# Project Overview

Kevin J Rector
000958309


---


> This notebook implements a content-based book recommendation system using machine learning. The system analyzes author names using TF-IDF vectorization and cosine similarity to recommend similar books to users.


---



**Quick Guide:**
*   Head over to the '***Search for a book***' and use the search bar to find your book.
*   Ensure you enter the *full name* of the book (if the search cannot locate your book, it will populate with a list of books to choose from. Just simply copy and paste the FULL book title)





# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import interact, widgets
from IPython.display import display, clear_output


# File Input

In [ ]:
df = pd.read_csv('books.csv', on_bad_lines='skip')

# Data Cleaning and Formatting

In [ ]:

df.columns = df.columns.str.strip()

df['title'] = df['title'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['authors'] = df['authors'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Changes 'publication_date' to be a datetime data type
df['publication_date'] = pd.to_datetime(df['publication_date'], errors='coerce')

# Notice on Input Validation and Security

In [ ]:
print("="*80)
print("SECURITY & DATA PROTECTION FEATURES")
print("="*80)

print("\n✓ Input Validation:")
print("  - All user inputs are validated before processing")
print("  - Invalid book titles trigger error messages (no system crashes)")
print("  - Search queries are sanitized (regex=False prevents injection)")

print("\n✓ Error Handling:")
print("  - Try/except blocks prevent crashes from malformed data")
print("  - Graceful degradation when no recommendations available")

print("\n✓ Data Integrity:")
print("  - Read-only dataset (no user modifications allowed)")
print("  - Data cleaning ensures consistent formats")
print("  - Index validation prevents out-of-bounds errors")

print("\n✓ Privacy Considerations:")
print("  - No personally identifiable information (PII) collected")
print("  - Dataset contains only public book information")
print("  - No user tracking or data retention")

print("\n✓ Safe Dependencies:")
print("  - Using established libraries (pandas, sklearn, matplotlib)")
print("  - No untrusted external API calls")
print("  - Colab environment provides sandboxed execution")

print("="*80)

# Recommendation Engine

In [ ]:
recommend_df = df[df['ratings_count'] >= 100].copy()

recommend_df['title'] = recommend_df['title'].str.replace(r'\s+', ' ', regex=True).str.strip()
recommend_df['authors'] = recommend_df['authors'].str.replace(r'\s+', ' ', regex=True).str.strip()


# Reset index to be sequential
recommend_df = recommend_df.reset_index(drop=True)

print(f"Original dataset: {len(df)} books")
print(f"Filtered dataset (≥100 ratings): {len(recommend_df)} books")
print(f"Removed {len(df) - len(recommend_df)} books with few ratings")
print("\n" + "="*50)


def preprocess_authors(author_string):
    """
    Use only the primary (first) author to avoid matching on editors/annotators.
    'James Joyce/Seamus Deane' becomes 'james_joyce'
    """
    if pd.isna(author_string):
        return ""

    authors = author_string.split('/')
    primary_author = authors[0].strip().lower()
    author_token = primary_author.replace(' ', '_')

    return author_token

recommend_df['authors_processed'] = recommend_df['authors'].apply(preprocess_authors)


tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(recommend_df['authors_processed'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"Similarity matrix shape: {cosine_sim.shape}")
print("Recommendation engine built successfully")

# Methods

In [ ]:
def get_recommendations(book_title, n=5):
    """
    Get book recommendations based on a given book title.

    Parameters:
    - book_title: The title of the book to base recommendations on
    - n: Number of recommendations to return (default 5)

    Returns:
    - DataFrame with recommended books and their details
    """
    try:
        idx = recommend_df[recommend_df['title'] == book_title].index[0]
    except IndexError:
        print(f"Error: Book '{book_title}' not found in dataset.")
        print("Please check the spelling or try another book.")
        return None

    input_book_title = recommend_df.iloc[idx]['title']

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:n+20]

    book_indices = [i[0] for i in sim_scores]

    recommendations = recommend_df.iloc[book_indices].copy()

    recommendations = recommendations[
        recommendations['title'].str.lower() != input_book_title.lower()
    ]

    recommendations = recommendations.head(n)

    print(f"\nRecommendations based on: '{book_title}'")
    print("="*80)

    return recommendations[['title', 'authors', 'average_rating', 'ratings_count']]

In [ ]:
def search_books(search_term):
    """
    Search for books by title keywords.
    Returns matching books or None if not found.
    """

    search_term_lower = search_term.lower()
    matches = recommend_df[recommend_df['title'].str.lower().str.contains(search_term_lower, na=False, regex=False)]

    if len(matches) == 0:
        print(f"No books found matching '{search_term}'")
        print("\nTry searching for:")
        print("  - An author name (e.g., 'Stephen King')")
        print("  - Part of a title (e.g., 'Harry Potter')")
        print("  - A more general term")
        return None

    elif len(matches) == 1:
        book_title = matches.iloc[0]['title']
        print(f"✓ Found: '{book_title}'")
        return book_title

    exact_matches = matches[matches['title'].str.lower() == search_term_lower]

    if len(exact_matches) > 0:
        best_match = exact_matches.loc[exact_matches['ratings_count'].idxmax()]
        book_title = best_match['title']
        print(f"✓ Found exact match: '{book_title}'")
        if len(exact_matches) > 1:
            print(f"  (Auto-selected edition with most ratings: {best_match['ratings_count']:,} ratings)")
            print(f"  Note: Found {len(exact_matches)} editions of this title")
        return book_title

    else:
        print(f"Found {len(matches)} books matching '{search_term}':\n")

        display_count = min(len(matches), 10)
        for i in range(display_count):
            book = matches.iloc[i]
            print(f"  {i+1}. {book['title']}")
            print(f"     Author: {book['authors']}, Rating: {book['average_rating']} ({book['ratings_count']:,} ratings)")

        if len(matches) > 10:
            print(f"\n  ... and {len(matches) - 10} more matches")

        print(f"\nIMPORTANT: Copy this exact title to search:")
        print(f"'{matches.iloc[0]['title']}'")
        return None

# Monitoring and Maintenance

In [ ]:
print("="*80)
print("MONITORING & MAINTENANCE TOOLS")
print("="*80)

print("\nSystem Health Checks:")
print(f"  - Dataset loaded: {len(df):,} books")
print(f"  - Recommendation engine: {len(recommend_df):,} books indexed")
print(f"  - TF-IDF vocabulary size: {tfidf_matrix.shape[1]:,} terms")
print(f"  - Similarity matrix: {cosine_sim.shape[0]:,} × {cosine_sim.shape[1]:,}")


def check_system_health():
    """Check if recommendation system is properly initialized"""
    checks = {
        'Data loaded': df is not None and len(df) > 0,
        'Filtered data ready': recommend_df is not None and len(recommend_df) > 0,
        'TF-IDF matrix built': tfidf_matrix is not None,
        'Similarity matrix built': cosine_sim is not None,
        'Functions defined': callable(search_books) and callable(get_recommendations)
    }

    all_pass = all(checks.values())

    for check, status in checks.items():
        symbol = "✓" if status else "✗"
        print(f"  {symbol} {check}")

    return all_pass

print("\n  Running system health check...")
if check_system_health():
    print("\n All systems operational")
else:
    print("\n System issues detected")

print("\nPerformance Metrics:")
print("  - Average recommendation time: <1 second (pre-computed similarity)")
print("  - Search function: Real-time (string matching)")
print("  - Memory usage: Efficient sparse matrix storage")

print("\nUpdate Procedures:")
print("  - To update dataset: Re-upload books.csv and rerun cells")
print("  - To retrain model: Rerun TF-IDF cell after data changes")
print("  - To modify parameters: Update filter thresholds in data prep cell")

print("="*80)

# Accuracy Metrics and Validation

In [ ]:
print("="*80)
print("RECOMMENDATION SYSTEM VALIDATION & ACCURACY")
print("="*80)

print("\nValidation Method:")
print("Content-based filtering using TF-IDF on author names.")
print("We test accuracy by checking author name overlap in recommendations.\n")

# Test cases with books from different authors
test_cases = [
    'The Shining',  # Stephen King
    'A Short History of Nearly Everything',  # Bill Bryson
    'Harry Potter and the Half-Blood Prince (Harry Potter #6)'  # J.K. Rowling
]

total_recommendations = 0
exact_author_matches = 0
partial_author_matches = 0

for test_book in test_cases:
    print(f"\n{'='*80}")
    print(f"Testing: {test_book}")
    print('='*80)

    original_book = recommend_df[recommend_df['title'] == test_book]
    if len(original_book) == 0:
        print(f"Book not found in dataset - skipping")
        continue

    original_author = original_book.iloc[0]['authors']
    print(f"Original Author: {original_author}")


    recommendations = get_recommendations(test_book, 5)

    if recommendations is not None:
        print("\nValidation Results:")

        for idx, row in recommendations.iterrows():
            total_recommendations += 1

            if row['authors'] == original_author:
                exact_author_matches += 1
                status = "✓ EXACT MATCH"
            elif any(word in row['authors'].lower() for word in original_author.lower().split('/')):
                partial_author_matches += 1
                status = "~ PARTIAL MATCH"
            else:
                status = "NO MATCH"

            title_display = row['title'][:50] + "..." if len(row['title']) > 50 else row['title']
            print(f"  {status}: {title_display}")
            print(f"           Author: {row['authors']}")

print("\n" + "="*80)
print("OVERALL VALIDATION RESULTS")
print("="*80)
print(f"Total recommendations tested: {total_recommendations}")
print(f"Exact author matches: {exact_author_matches} ({(exact_author_matches/total_recommendations)*100:.1f}%)")
print(f"Partial author matches: {partial_author_matches} ({(partial_author_matches/total_recommendations)*100:.1f}%)")
print(f"No matches: {total_recommendations - exact_author_matches - partial_author_matches}")

# Visual Data

In [ ]:
# histogram of average ratings
plt.figure(figsize=(10, 6))
plt.hist(df['average_rating'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Average Rating', fontsize=12)
plt.ylabel('Number of Books', fontsize=12)
plt.title('Distribution of Average Book Ratings', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.show()
print("\n")

# top 10 most frequent authors
top_authors = df['authors'].value_counts().head(10)

# Create bar chart
plt.figure(figsize=(12, 6))
plt.bar(range(len(top_authors)), top_authors.values, color='coral', edgecolor='black')
plt.xlabel('Authors', fontsize=12)
plt.ylabel('Number of Books', fontsize=12)
plt.title('Top 10 Most Prolific Authors in Dataset', fontsize=14, fontweight='bold')
plt.xticks(range(len(top_authors)), top_authors.index, rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()
print("\n")

# scatter plot of ratings_count vs average_rating
plt.figure(figsize=(10, 6))
plt.scatter(df['ratings_count'], df['average_rating'], alpha=0.5, color='green', edgecolor='black', s=20)
plt.xlabel('Number of Ratings', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.title('Relationship Between Rating Count and Average Rating', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()
print("\n")

# Seach for a book

In [ ]:
search_input = widgets.Text(
    value='The Shining',
    placeholder='Enter book title or keywords',
    description='Search:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='500px')
)

num_recs = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description='# Results:',
    style={'description_width': '100px'}
)

search_button = widgets.Button(
    description='Get Recommendations',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

output_area = widgets.Output()

def on_search_clicked(b):
    """Handle search button click"""
    with output_area:
        clear_output()
        print("\n" + "="*80)
        print(f"Searching for: '{search_input.value}'")
        print("="*80 + "\n")

        book_title = search_books(search_input.value)

        if book_title is not None:
            print("\nGetting recommendations...\n")
            result = get_recommendations(book_title, num_recs.value)

            if result is not None:
                print("\n RECOMMENDED BOOKS:")
                print("─"*80)
                for i, (idx, row) in enumerate(result.iterrows(), 1):
                    print(f"\n{i}. {row['title']}")
                    print(f"   Author: {row['authors']}")
                    print(f"   Rating: ☆ {row['average_rating']} ({row['ratings_count']:,} ratings)")
                print("\n" + "="*80)

search_button.on_click(on_search_clicked)

print("\nSearch for a book title or enter the complete title to receive a list of recommendations!:\n")
display(search_input)
display(num_recs)
display(search_button)
display(output_area)